In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install turicreate
import turicreate

In [ ]:
products = turicreate.SFrame('../input/amazon-babygl/amazon_baby.gl')
products

In [ ]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

In [ ]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [ ]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'] )

In [ ]:
# Loop through word counts to create a classifier for only a few words 
# Created an individual column for each item 
for word in selected_words:
    products[word] = products['word_count'].apply(lambda counts: counts.get(word, 0))

products

In [ ]:
for word in selected_words:
    print("\nThe number of times {} appears: {}".format(word, products[word].sum()))

In [ ]:
train_data,test_data = products.random_split(.8, seed=0)

In [ ]:
# Features to be trained on 
features = selected_words

In [ ]:
products['rating'].show()

In [ ]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [ ]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [ ]:
products

In [ ]:
products['sentiment'].show()

In [ ]:
train_data,test_data = products.random_split(.8,seed=0)

In [ ]:
# Original Analysis Model 
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', 
                                                        features=['word_count'], 
                                                        validation_set=test_data)

In [ ]:
# Creating the model with selected words 
selected_words_model = turicreate.logistic_classifier.create(train_data,target='sentiment', 
                                                        features=features, 
                                                        validation_set=test_data)

In [ ]:
# Calling and describing our coefficients and weights allotted to each word
selected_words_model.coefficients.sort(key_column_names='value', ascending=True)

In [ ]:
# Evaluate the orginal analysis model first 
sentiment_model.evaluate(test_data)

In [ ]:
# Evaluate the limited words model 
selected_words_model.evaluate(test_data)

In [ ]:
# Extract only the relevant data
diaper_champ_reviews = products[products['name']== 'Baby Trend Diaper Champ']
diaper_champ_reviews

In [ ]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

In [ ]:
diaper_champ_reviews['predicted_sentiment'] = selected_words_model.predict(diaper_champ_reviews, output_type = 'probability')

In [ ]:
products

In [ ]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)
diaper_champ_reviews

In [ ]:
diaper_champ_reviews.tail()

In [ ]:
diaper_champ_reviews[0]['review']


In [ ]:
diaper_champ_reviews[-1]['review']
